In [135]:
# IMPORTANT: if this document does not compile with your default data rate limit, open jupyter notebook using the 
# following command: jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

import sqlite3
import pandas as pd
from Bio import Entrez
from Bio import SeqIO
import sys


Entrez.email = "thomas.galeon@berkeley.edu"

# Creating 3 SQL tables
conn = sqlite3.connect('Lab4.db')
c = conn.cursor()

# The previous SQL tables are erased if they exist. They will be recreated and populated automatically.
try:
    c.execute("DROP TABLE genes")
    c.execute("DROP TABLE pathways")
    c.execute("DROP TABLE enzymes")
    c.execute("DROP TABLE pathwayenzyme")
    c.execute("DROP TABLE enzymegene")
except:
    print("Non existent tables. Tables will be created now.")
    
# 5 SQL Tables are created - the detailed description of these tables can be found in the README file    
c.execute("""CREATE TABLE genes (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                name TEXT,
                                organism TEXT,
                                sequence TEXT);""")


c.execute("""CREATE TABLE pathways (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                    name TEXT,
                                    description TEXT);""")

c.execute("""CREATE TABLE enzymes (ec TEXT PRIMARY KEY,
                                    name TEXT,
                                    description TEXT);""")

c.execute("""CREATE TABLE pathwayenzyme (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                            pathway TEXT,
                                            enzyme TEXT,
                                            pathway_order INTEGER);""")


c.execute("""CREATE TABLE enzymegene (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                            ec TEXT,
                                            organism TEXT,
                                            gene TEXT);""")



# Considered organisms, pathways and enzymes. Enzymes are given in order of occurence in the pathway.
org = ['homo sapiens', 'escherichia coli', 'drosophila']



pwt = {'Glycolysis': ['4.1.1.49','5.3.1.9', '5.4.2.2', '1.8.1.4'], 
      'TCA cycle':['4.1.1.31','6.2.1.4','1.1.5.4','4.1.1.49',],
      'Pentose Phosphate': ['3.1.3.11','2.7.1.15', '3.1.1.31','5.4.2.7']}


#'TCA cycle':['1.2.4.1', '4.2.1.3', '6.2.1.5', '2.3.1.61'], ['1.3.5.4', '4.1.1.31', '1.1.1.37', '2.3.3.1', '2.3.1.12', '6.4.1.1', '1.1.1.42'
#'1.3.5.4', '1.2.4.2', '1.1.1.286', '6.2.1.4', '1.2.7.11' '1.2.4.2',, '6.2.1.4', '1.2.7.11'
# The pathway description table is populated manually. The sources for the pathway descriptions are 
#indicated in the README file.

c.execute("""INSERT INTO pathways (name,
                                    description)
                            VALUES
                                    ('Glycolysis',
                                    'Glycolysis is the process of converting glucose into pyruvate and generating small 
                                    amounts of ATP (energy) and NADH (reducing power). It is a central pathway that 
                                    produces important precursor metabolites: six-carbon compounds of glucose-6P and 
                                    fructose-6P and three-carbon compounds of glycerone-P, glyceraldehyde-3P, 
                                    glycerate-3P, phosphoenolpyruvate, and pyruvate. Acetyl-CoA, another important 
                                    precursor metabolite, is produced by oxidative decarboxylation of pyruvate. When the
                                    enzyme genes of this pathway are examined in completely sequenced genomes, the 
                                    reaction steps of three-carbon compounds from glycerone-P to pyruvate form a 
                                    conserved core module, which is found in almost all organisms and which sometimes
                                    contains operon structures in bacterial genomes. Gluconeogenesis is a synthesis 
                                    pathway of glucose from noncarbohydrate precursors. It is essentially a reversal of 
                                    glycolysis with minor variations of alternative paths.')
                                    ;""")


c.execute("""INSERT INTO pathways (name,
                                    description)
                            VALUES
                                    ('TCA cycle',
                                    'The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the 
                                    final steps of the oxidation of carbohydrates and fatty acids. The cycle starts with 
                                    acetyl-CoA, the activated form of acetate, derived from glycolysis and pyruvate 
                                    oxidation for carbohydrates and from beta oxidation of fatty acids. The two-carbon 
                                    acetyl group in acetyl-CoA is transferred to the four-carbon compound of oxaloacetate 
                                    to form the six-carbon compound of citrate. In a series of reactions two carbons in 
                                    citrate are oxidized to CO2 and the reaction pathway supplies NADH for use in the 
                                    oxidative phosphorylation and other metabolic processes. The pathway also supplies 
                                    important precursor metabolites including 2-oxoglutarate. At the end of the cycle the 
                                    remaining four-carbon part is transformed back to oxaloacetate. According to the 
                                    genome sequence data, many organisms seem to lack genes for the full cycle, but 
                                    contain genes for specific segments.')
                                    ;""")

c.execute("""INSERT INTO pathways (name,
                                    description)
                            VALUES
                                    ('Pentose Phosphate',
                                    'The pentose phosphate pathway is a process of glucose turnover that produces NADPH 
                                    as reducing equivalents and pentoses as essential parts of nucleotides. There are two 
                                    different phases in the pathway. One is irreversible oxidative phase in which 
                                    glucose-6P is converted to ribulose-5P by oxidative decarboxylation, and NADPH is 
                                    generated. The other is reversible non-oxidative phase in which phosphorylated sugars 
                                    are interconverted to generate xylulose-5P, ribulose-5P, and ribose-5P. 
                                    Phosphoribosyl pyrophosphate (PRPP) formed from ribose-5P is an activated compound 
                                    used in the biosynthesis of histidine and purine/pyrimidine nucleotides. This pathway 
                                    map also shows the Entner-Doudoroff pathway where 6-P-gluconate is dehydrated and then
                                    cleaved into pyruvate and glyceraldehyde-3P.')
                                    ;""")



# The other tables are populated automatically by extracting the data from Entrez
for (path,enz) in pwt.items():
    # Order is the number of the enzyme in the pathway
    order = 1
    for e in enz:
        # The asscoiated table linking pathways and enzymes is created
        c.execute("""INSERT INTO pathwayenzyme (pathway,
                                                enzyme,
                                                pathway_order)
                                        VALUES
                                                (?,
                                                ?,
                                                ?);""" ,
                                                (path,e,order))
        order +=1
             
       
        
        # The organism specific sequence is determined here
        for o in org:
            # The current search is printed to give the user an indication of progress
            print('Pathway: ' + path + '; EC: ' + e + '; Organism: ' + o)

             # The corresponding protein information is searched for using Entrez
            handle = Entrez.esearch(db = 'protein',
                                    term = 'EC: ' + e + ' AND ' + o,
                                    sort = 'relevance',
                                    idtype = 'acc',
                                    retmax = 1)

            for i in Entrez.read(handle)['IdList']:
                handle = Entrez.efetch(db = 'protein', id = i, rettype = 'gb', retmode = 'text')
                record = SeqIO.read(handle, "genbank")
                handle.close()
                c.execute("""INSERT OR REPLACE INTO enzymes (name,
                                                description,
                                                ec)
                                        VALUES
                                                (?,
                                                ?,
                                                ?);""" ,
                                                (record.name,record.description,e))
                
            # The corresponding nucleotide sequence information is searched for using Entrez
            # The sequence is found using the gene database, the rest of the information is found using the nucleotide one
            handle = Entrez.esearch(db = 'gene',
                                    term = e + ' ' + o,
                                    sort = 'relevance',
                                    idtype = 'acc',
                                    retmax = 1)
            
            
            for i in Entrez.read(handle)['IdList']:
                handle = Entrez.efetch(db = 'nucleotide', id = i, rettype = 'gb', retmode = 'text')
                record = SeqIO.read(handle, "genbank")
                handle.close()

                c.execute("""INSERT INTO genes (name,
                                                organism,
                                                sequence)
                                        VALUES
                                                (?,
                                                ?,
                                                ?);""" ,
                                                (record.id, o, str(record.seq)))
        
                c.execute("""INSERT INTO enzymegene (ec,
                                                    organism,
                                                    gene)
                                            VALUES
                                                    (?,
                                                    ?,
                                                    ?);""" ,
                                                    (e,o,record.id))
            

# Update tables
conn.commit()

# Print tables
print("\nPathways table:")
print (pd.read_sql_query("SELECT * FROM pathways", conn))
print("\nGenes table:")
print (pd.read_sql_query("SELECT * FROM genes", conn))
print("\nEnzymes table:")
print (pd.read_sql_query("SELECT * FROM enzymes", conn))
print("\nPathway enzyme associated table:")
print (pd.read_sql_query("SELECT * FROM pathwayenzyme", conn))
print("\nEnzyme genes associated table:")
print (pd.read_sql_query("SELECT * FROM enzymegene", conn))
                  




Non existent tables. Tables will be created now.
Pathway: Glycolysis; EC: 4.1.1.49; Organism: homo sapiens
Pathway: Glycolysis; EC: 4.1.1.49; Organism: escherichia coli
Pathway: Glycolysis; EC: 4.1.1.49; Organism: drosophila
Pathway: Glycolysis; EC: 5.3.1.9; Organism: homo sapiens
Pathway: Glycolysis; EC: 5.3.1.9; Organism: escherichia coli
Pathway: Glycolysis; EC: 5.3.1.9; Organism: drosophila
Pathway: Glycolysis; EC: 5.4.2.2; Organism: homo sapiens
Pathway: Glycolysis; EC: 5.4.2.2; Organism: escherichia coli
Pathway: Glycolysis; EC: 5.4.2.2; Organism: drosophila
Pathway: Glycolysis; EC: 1.8.1.4; Organism: homo sapiens
Pathway: Glycolysis; EC: 1.8.1.4; Organism: escherichia coli
Pathway: Glycolysis; EC: 1.8.1.4; Organism: drosophila
Pathway: TCA cycle; EC: 4.1.1.31; Organism: homo sapiens
Pathway: TCA cycle; EC: 4.1.1.31; Organism: escherichia coli
Pathway: TCA cycle; EC: 4.1.1.31; Organism: drosophila
Pathway: TCA cycle; EC: 6.2.1.4; Organism: homo sapiens
Pathway: TCA cycle; EC: 6.2